# Computing distance to line

An essential part of getting the robot to behave correctly around street lights is for it to know its distance to the light. On actual roads in Switzerland and many other countries, at every light there is a stop line painted on the road. We do the same in our context and thus the distance to the street light is equivalent to the distance to the stop line on the road.

Below is an example of a typical picture taken by the robot during one of its runs, where the stop line is starting to appear. The whole set of pictures from that run can be found in the folder /images/.

![title](images/27.jpg)

Write a function that takes an image as input and returns a distance to the line (not necessarily in cm). To make sure it is robust, find a way to augment your test set from the given images.

Use OpenCV, which you can install through Anaconda, to process the images. Then you can choose between using the OpenCV library to detect the line or implementing your own procedure, keeping in mind that the function should be as fast as possible. 